In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
print("⚠️  Dynamic Parallelism is a CUDA C++ feature (CC 3.5+)!")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## 1. What is Dynamic Parallelism?

```
Traditional CUDA:                  Dynamic Parallelism:
━━━━━━━━━━━━━━━━━━                 ━━━━━━━━━━━━━━━━━━━━
Host controls everything           Kernels launch kernels!

[Host] ──> [Kernel A]              [Host] ──> [Parent Kernel]
[Host] ──> [Kernel B]                            │
[Host] ──> [Kernel C]                            ├──> [Child 1]
                                                 ├──> [Child 2]
                                                 └──> [Child 3]
```

**Use Cases:**
- Recursive algorithms (quicksort, tree traversal)
- Adaptive mesh refinement
- Work that generates more work
- Data-dependent parallelism

---

## 2. Basic Child Kernel Launch

The simplest example: a parent kernel launches child kernels from the GPU.

In [ ]:
%%writefile dynamic_basic.cu
#include <cuda_runtime.h>
#include <stdio.h>

// ============================================
// Child Kernel - Launched FROM the GPU
// ============================================
__global__ void childKernel(int parentId) {
    int tid = threadIdx.x;
    printf("    Child of parent %d: thread %d\n", parentId, tid);
}

// ============================================
// Parent Kernel - Launches Children
// ============================================
__global__ void parentKernel() {
    int tid = threadIdx.x;
    
    printf("Parent thread %d launching child kernel...\n", tid);
    
    // Launch child kernel from GPU!
    // Same <<<blocks, threads>>> syntax
    childKernel<<<1, 4>>>(tid);
    
    // Device-side synchronization
    cudaDeviceSynchronize();
    
    printf("Parent thread %d: child completed\n", tid);
}

int main() {
    printf("=== Dynamic Parallelism Basic Demo ===\n\n");
    printf("Host launching parent kernel with 2 threads...\n\n");
    
    // Host launches parent
    parentKernel<<<1, 2>>>();
    
    // Host-side sync waits for ALL work (parent + children)
    cudaDeviceSynchronize();
    
    printf("\nAll done!\n");
    return 0;
}

In [ ]:
# Note: -rdc=true enables relocatable device code (required for dynamic parallelism)
# Note: -lcudadevrt links the device runtime library
!nvcc -rdc=true -lcudadevrt -o dynamic_basic dynamic_basic.cu && ./dynamic_basic

---

## 3. Memory Visibility Between Parent and Child

Understanding what memory is visible where is critical:

| Memory Type | Parent → Child | Child → Parent |
|-------------|----------------|----------------|
| **Global** | ✅ After launch | ✅ After sync |
| **Local** | ❌ | ❌ |
| **Shared** | ❌ | ❌ |
| **Constant** | ✅ (set before host launch) | ✅ |

In [ ]:
%%writefile dynamic_memory.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void childKernel(float* data, int idx) {
    // Child CAN read parent's global memory writes
    float parentValue = data[idx];
    printf("    Child[%d]: read %.1f from parent\n", idx, parentValue);
    
    // Child writes to global memory
    data[idx] = parentValue * 2.0f;
    printf("    Child[%d]: wrote %.1f back\n", idx, data[idx]);
}

__global__ void parentKernel(float* data) {
    int tid = threadIdx.x;
    
    // Parent writes to global memory
    data[tid] = (float)(tid + 1) * 10.0f;
    printf("Parent %d: wrote %.1f to global memory\n", tid, data[tid]);
    
    // __threadfence() ensures memory writes are visible
    // to child kernels BEFORE we launch them
    __threadfence();
    
    // Launch child
    childKernel<<<1, 1>>>(data, tid);
    
    // Must sync to see child's writes!
    cudaDeviceSynchronize();
    
    // Now parent can see child's writes
    printf("Parent %d: after sync, data = %.1f\n", tid, data[tid]);
}

int main() {
    printf("=== Dynamic Parallelism Memory Visibility ===\n\n");
    
    float* d_data;
    cudaMalloc(&d_data, 4 * sizeof(float));
    cudaMemset(d_data, 0, 4 * sizeof(float));
    
    parentKernel<<<1, 4>>>(d_data);
    cudaDeviceSynchronize();
    
    // Verify from host
    float h_data[4];
    cudaMemcpy(h_data, d_data, 4 * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("\nHost verification:\n");
    for (int i = 0; i < 4; i++) {
        float expected = (float)(i + 1) * 10.0f * 2.0f;  // parent wrote * child doubled
        printf("  data[%d] = %.1f (expected: %.1f)\n", i, h_data[i], expected);
    }
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o dynamic_memory dynamic_memory.cu && ./dynamic_memory

---

## 4. Device-Side Streams

Kernels on the GPU can create and use streams, just like on the host. This enables concurrent child kernel execution.

In [ ]:
%%writefile dynamic_streams.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void childA(int* counter, int id) {
    atomicAdd(counter, 1);
    printf("    ChildA[%d] executed, counter now %d\n", id, *counter);
}

__global__ void childB(int* counter, int id) {
    atomicAdd(counter, 10);
    printf("    ChildB[%d] executed, counter now %d\n", id, *counter);
}

__global__ void parentKernel(int* counter) {
    printf("=== Default Stream (Sequential) ===\n");
    
    // Default stream: children execute sequentially
    childA<<<1, 1>>>(counter, 0);  // First
    childB<<<1, 1>>>(counter, 0);  // Second (waits for childA)
    
    cudaDeviceSynchronize();
    printf("After default stream: counter = %d\n\n", *counter);
    
    printf("=== Named Streams (Concurrent) ===\n");
    
    // Create device-side streams
    cudaStream_t stream1, stream2;
    cudaStreamCreateWithFlags(&stream1, cudaStreamNonBlocking);
    cudaStreamCreateWithFlags(&stream2, cudaStreamNonBlocking);
    
    // Launch children on different streams - can run concurrently!
    childA<<<1, 1, 0, stream1>>>(counter, 1);
    childB<<<1, 1, 0, stream2>>>(counter, 1);
    
    // Sync each stream
    cudaStreamSynchronize(stream1);
    cudaStreamSynchronize(stream2);
    
    printf("After named streams: counter = %d\n", *counter);
    
    // Cleanup device-side streams
    cudaStreamDestroy(stream1);
    cudaStreamDestroy(stream2);
}

int main() {
    printf("=== Device-Side Streams Demo ===\n\n");
    
    int* d_counter;
    cudaMalloc(&d_counter, sizeof(int));
    cudaMemset(d_counter, 0, sizeof(int));
    
    parentKernel<<<1, 1>>>(d_counter);
    cudaDeviceSynchronize();
    
    int result;
    cudaMemcpy(&result, d_counter, sizeof(int), cudaMemcpyDeviceToHost);
    printf("\nFinal counter value: %d (expected: 22 = 1+10+1+10)\n", result);
    
    cudaFree(d_counter);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o dynamic_streams dynamic_streams.cu && ./dynamic_streams

---

## 5. Synchronization Patterns

There are several ways to synchronize parent with child kernels.

In [ ]:
%%writefile sync_patterns.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void workKernel(float* data, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        data[i] = data[i] * 2.0f;
    }
}

// Pattern 1: Explicit sync before reading results
__global__ void pattern1_explicit_sync(float* data, int n) {
    printf("Pattern 1: Explicit sync\n");
    
    workKernel<<<(n + 255) / 256, 256>>>(data, n);
    
    // REQUIRED before reading child's results
    cudaDeviceSynchronize();
    
    printf("  First result: %.1f\n", data[0]);
}

// Pattern 2: Fire and forget (no parent sync)
__global__ void pattern2_fire_forget(float* data, int n) {
    printf("Pattern 2: Fire and forget\n");
    
    workKernel<<<(n + 255) / 256, 256>>>(data, n);
    
    // NO sync - parent returns immediately
    // Child continues running
    // Result visible to HOST after host's cudaDeviceSynchronize()
    
    printf("  Parent returning (child may still be running)\n");
}

// Pattern 3: Stream-based sync (more granular control)
__global__ void pattern3_stream_sync(float* data, int n) {
    printf("Pattern 3: Stream-based sync\n");
    
    cudaStream_t stream;
    cudaStreamCreateWithFlags(&stream, cudaStreamNonBlocking);
    
    workKernel<<<(n + 255) / 256, 256, 0, stream>>>(data, n);
    
    // Sync only this stream
    cudaStreamSynchronize(stream);
    
    printf("  First result: %.1f\n", data[0]);
    
    cudaStreamDestroy(stream);
}

int main() {
    printf("=== Synchronization Patterns ===\n\n");
    
    const int N = 1024;
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Initialize to 1.0
    float h_init[N];
    for (int i = 0; i < N; i++) h_init[i] = 1.0f;
    
    // Test Pattern 1
    cudaMemcpy(d_data, h_init, N * sizeof(float), cudaMemcpyHostToDevice);
    pattern1_explicit_sync<<<1, 1>>>(d_data, N);
    cudaDeviceSynchronize();
    printf("\n");
    
    // Test Pattern 2
    cudaMemcpy(d_data, h_init, N * sizeof(float), cudaMemcpyHostToDevice);
    pattern2_fire_forget<<<1, 1>>>(d_data, N);
    cudaDeviceSynchronize();  // Host waits for all
    float result;
    cudaMemcpy(&result, d_data, sizeof(float), cudaMemcpyDeviceToHost);
    printf("  (Host sees result: %.1f)\n\n", result);
    
    // Test Pattern 3
    cudaMemcpy(d_data, h_init, N * sizeof(float), cudaMemcpyHostToDevice);
    pattern3_stream_sync<<<1, 1>>>(d_data, N);
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o sync_patterns sync_patterns.cu && ./sync_patterns

---

## 6. Error Handling in Dynamic Parallelism

Device-side error checking works similarly to host-side.

In [ ]:
%%writefile dynamic_errors.cu
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void simpleChild(float* data) {
    data[threadIdx.x] = 1.0f;
}

__global__ void errorCheckingParent(float* data) {
    printf("=== Device-Side Error Checking ===\n\n");
    
    // Attempt 1: Valid launch
    printf("Attempt 1: Valid launch (1 block, 32 threads)\n");
    simpleChild<<<1, 32>>>(data);
    
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("  Launch error: %s\n", cudaGetErrorString(err));
    } else {
        printf("  Launch successful!\n");
    }
    
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        printf("  Execution error: %s\n", cudaGetErrorString(err));
    } else {
        printf("  Execution successful!\n");
    }
    
    // Attempt 2: Invalid launch (too many threads)
    printf("\nAttempt 2: Invalid launch (1 block, 2048 threads)\n");
    simpleChild<<<1, 2048>>>(data);  // Exceeds max threads per block!
    
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("  Launch error: %s\n", cudaGetErrorString(err));
    } else {
        printf("  Launch seemed OK (checking execution...)\n");
        err = cudaDeviceSynchronize();
        if (err != cudaSuccess) {
            printf("  Execution error: %s\n", cudaGetErrorString(err));
        }
    }
    
    // Attempt 3: Another valid launch to show recovery
    printf("\nAttempt 3: Recovery - valid launch again\n");
    simpleChild<<<1, 32>>>(data);
    err = cudaDeviceSynchronize();
    printf("  Result: %s\n", err == cudaSuccess ? "Success!" : cudaGetErrorString(err));
}

int main() {
    float* d_data;
    cudaMalloc(&d_data, 2048 * sizeof(float));
    
    errorCheckingParent<<<1, 1>>>(d_data);
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o dynamic_errors dynamic_errors.cu && ./dynamic_errors

---

## 7. Practical Example: Recursive Sum

A classic example: recursively sum an array by dividing work.

In [ ]:
%%writefile recursive_sum.cu
#include <cuda_runtime.h>
#include <stdio.h>

#define MAX_DEPTH 4

// Recursive kernel that sums array segments
__global__ void recursiveSum(float* data, float* result, int start, int end, int depth) {
    int n = end - start;
    
    // Base case: small enough to sum directly
    if (n <= 32 || depth >= MAX_DEPTH) {
        float sum = 0.0f;
        for (int i = start; i < end; i++) {
            sum += data[i];
        }
        atomicAdd(result, sum);
        return;
    }
    
    // Recursive case: split and launch children
    int mid = start + n / 2;
    
    // Launch two children for each half
    recursiveSum<<<1, 1>>>(data, result, start, mid, depth + 1);
    recursiveSum<<<1, 1>>>(data, result, mid, end, depth + 1);
    
    // Wait for children
    cudaDeviceSynchronize();
}

int main() {
    printf("=== Recursive Sum with Dynamic Parallelism ===\n\n");
    
    const int N = 1024;
    float* d_data;
    float* d_result;
    
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_result, sizeof(float));
    
    // Initialize: 1, 2, 3, ..., N
    float* h_data = (float*)malloc(N * sizeof(float));
    float expected = 0.0f;
    for (int i = 0; i < N; i++) {
        h_data[i] = (float)(i + 1);
        expected += h_data[i];
    }
    
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemset(d_result, 0, sizeof(float));
    
    printf("Summing 1 + 2 + 3 + ... + %d\n", N);
    printf("Expected: %.0f\n\n", expected);
    
    // Launch recursive sum
    recursiveSum<<<1, 1>>>(d_data, d_result, 0, N, 0);
    cudaDeviceSynchronize();
    
    // Get result
    float result;
    cudaMemcpy(&result, d_result, sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("Result: %.0f\n", result);
    printf("Match: %s\n", (result == expected) ? "YES" : "NO");
    
    cudaFree(d_data);
    cudaFree(d_result);
    free(h_data);
    
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o recursive_sum recursive_sum.cu && ./recursive_sum

---

## 8. Practical Example: Adaptive Work Generation

Only launch child kernels where work is needed (data-dependent parallelism).

In [ ]:
%%writefile adaptive_work.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Child kernel: process a segment that needs work
__global__ void processSegment(float* data, int start, int count, int* processed) {
    int i = start + blockIdx.x * blockDim.x + threadIdx.x;
    if (i < start + count) {
        // Simulate processing: square the value
        data[i] = data[i] * data[i];
        atomicAdd(processed, 1);
    }
}

// Parent: analyze data and launch children only where needed
__global__ void adaptiveParent(float* data, int n, float threshold, int* processed) {
    int tid = threadIdx.x;
    int segments = 8;  // Divide into 8 segments
    int segmentSize = n / segments;
    
    if (tid < segments) {
        int start = tid * segmentSize;
        int end = (tid == segments - 1) ? n : start + segmentSize;
        int count = end - start;
        
        // Check if this segment needs processing
        // (contains values above threshold)
        bool needsWork = false;
        for (int i = start; i < end && !needsWork; i++) {
            if (data[i] > threshold) needsWork = true;
        }
        
        if (needsWork) {
            printf("Segment %d [%d-%d]: launching child kernel\n", tid, start, end - 1);
            
            int threads = 256;
            int blocks = (count + threads - 1) / threads;
            processSegment<<<blocks, threads>>>(data, start, count, processed);
        } else {
            printf("Segment %d [%d-%d]: skipping (no work needed)\n", tid, start, end - 1);
        }
    }
    
    // Sync all children
    __syncthreads();
    if (tid == 0) {
        cudaDeviceSynchronize();
    }
}

int main() {
    printf("=== Adaptive Work Generation ===\n\n");
    
    const int N = 8192;
    const float THRESHOLD = 0.5f;
    
    float* d_data;
    int* d_processed;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_processed, sizeof(int));
    cudaMemset(d_processed, 0, sizeof(int));
    
    // Initialize: some segments have values > threshold, some don't
    float* h_data = (float*)malloc(N * sizeof(float));
    for (int i = 0; i < N; i++) {
        // Segments 0,2,4,6 have low values; 1,3,5,7 have high values
        int segment = i / (N / 8);
        h_data[i] = (segment % 2 == 0) ? 0.3f : 0.7f;
    }
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    
    printf("Threshold: %.1f\n", THRESHOLD);
    printf("Only segments with values > threshold will be processed.\n\n");
    
    adaptiveParent<<<1, 8>>>(d_data, N, THRESHOLD, d_processed);
    cudaDeviceSynchronize();
    
    int processed;
    cudaMemcpy(&processed, d_processed, sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("\nElements processed: %d (out of %d)\n", processed, N);
    printf("Expected: %d (half of elements)\n", N / 2);
    
    cudaFree(d_data);
    cudaFree(d_processed);
    free(h_data);
    
    return 0;
}

In [ ]:
!nvcc -rdc=true -lcudadevrt -o adaptive_work adaptive_work.cu && ./adaptive_work

---

## Summary: Dynamic Parallelism Key Points

### Compilation
```bash
nvcc -rdc=true -lcudadevrt -o program program.cu
```

### Launch Syntax (Same as Host)
```cpp
childKernel<<<blocks, threads, sharedMem, stream>>>(args);
```

### Synchronization
```cpp
cudaDeviceSynchronize();      // Wait for all children
cudaStreamSynchronize(s);     // Wait for specific stream
```

### Memory Visibility
- Global memory: visible after sync
- Local/Shared: NOT visible between parent/child
- Use `__threadfence()` before launching children

### Limits
- Maximum nesting depth: 24 levels
- Compute Capability: 3.5+
- Child kernel launches have overhead

---

## Exercises

### Exercise 1: Recursive Quicksort
Implement quicksort using dynamic parallelism - each partition launches children.

### Exercise 2: Tree Traversal
Process a tree structure where each node can launch children for its subtrees.

### Exercise 3: Adaptive Grid Refinement
Implement a 2D grid where cells above a threshold launch finer grids.

In [ ]:
# Cleanup generated files
!rm -f dynamic_basic dynamic_memory dynamic_streams sync_patterns dynamic_errors recursive_sum adaptive_work *.cu

---

## Part 1: What is Dynamic Parallelism?

### Concept

```
Traditional CUDA:
━━━━━━━━━━━━━━━━━━
Host (CPU) launches kernels on Device (GPU)

[Host] ──launch──> [Kernel A]
[Host] ──launch──> [Kernel B]
[Host] ──launch──> [Kernel C]

Dynamic Parallelism:
━━━━━━━━━━━━━━━━━━━━━
GPU kernels can launch other kernels!

[Host] ──launch──> [Kernel A]
                      │
                      ├──launch──> [Child 1]
                      ├──launch──> [Child 2]
                      └──launch──> [Child 3]

Use Cases:
• Recursive algorithms (quicksort, tree traversal)
• Adaptive mesh refinement
• Work that generates more work
• Data-dependent parallelism
```

---

## Part 2: Basic Child Kernel Launch

### 🔷 CUDA C++ Dynamic Parallelism (Primary)

**Compile with relocatable device code:**
```bash
nvcc -arch=sm_75 -rdc=true -lcudadevrt -o dynamic_basic dynamic_basic.cu
```

In [ ]:
%%writefile dynamic_basic.cu
// dynamic_basic.cu - Basic child kernel launch
#include <stdio.h>
#include <cuda_runtime.h>

// ============================================
// Child Kernel - Launched from GPU
// ============================================
__global__ void childKernel(int parentId) {
    int tid = threadIdx.x;
    printf("  Child of parent %d: thread %d\n", parentId, tid);
}

// ============================================
// Parent Kernel - Launches Children
// ============================================
__global__ void parentKernel() {
    int tid = threadIdx.x;
    
    printf("Parent thread %d launching child...\n", tid);
    
    // Launch child kernel from GPU!
    childKernel<<<1, 4>>>(tid);
    
    // Wait for child to complete
    cudaDeviceSynchronize();  // Device-side sync!
    
    printf("Parent thread %d: child completed\n", tid);
}

int main() {
    printf("Host launching parent kernel...\n");
    
    parentKernel<<<1, 2>>>();
    cudaDeviceSynchronize();  // Host-side sync
    
    printf("All done!\n");
    return 0;
}

---

## Part 3: Memory Visibility

### Memory Rules for Dynamic Parallelism

```
┌─────────────────────────────────────────────────────────┐
│           MEMORY VISIBILITY RULES                       │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Global Memory:                                         │
│  ✅ Parent writes visible to child after launch         │
│  ✅ Child writes visible to parent after sync           │
│  ⚠️  Must sync properly!                                │
│                                                         │
│  Local Memory:                                          │
│  ❌ Parent's local memory NOT visible to child          │
│  ❌ Child's local memory NOT visible to parent          │
│                                                         │
│  Shared Memory:                                         │
│  ❌ Parent's shared NOT visible to child                │
│  ❌ Each kernel has its own shared memory               │
│                                                         │
│  Constant Memory:                                       │
│  ✅ Visible to all (set before host launch)             │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

### 🔷 CUDA C++ Memory Example (Primary)

```cpp
// dynamic_memory.cu - Memory visibility
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void childKernel(float* data, int idx) {
    // Child can read parent's global memory writes
    float parentValue = data[idx];
    printf("Child: read %.1f from parent\n", parentValue);
    
    // Child writes to global memory
    data[idx] = parentValue * 2.0f;
}

__global__ void parentKernel(float* data) {
    int tid = threadIdx.x;
    
    // Parent writes to global memory
    data[tid] = (float)(tid + 1) * 10.0f;
    printf("Parent %d: wrote %.1f\n", tid, data[tid]);
    
    // __threadfence() ensures visibility before child launch
    __threadfence();
    
    // Launch child
    childKernel<<<1, 1>>>(data, tid);
    
    // Must sync to see child's writes!
    cudaDeviceSynchronize();
    
    // Now parent can see child's writes
    printf("Parent %d: child wrote %.1f\n", tid, data[tid]);
}

int main() {
    float* d_data;
    cudaMalloc(&d_data, 4 * sizeof(float));
    
    parentKernel<<<1, 4>>>(d_data);
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    return 0;
}
```

---

## Part 4: Device-Side Streams

### 🔷 CUDA C++ Streams in Dynamic Parallelism (Primary)

```cpp
// dynamic_streams.cu - Device-side streams
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void childA(int* data) {
    atomicAdd(data, 1);
}

__global__ void childB(int* data) {
    atomicAdd(data, 10);
}

__global__ void parentKernel(int* data) {
    // ============================================
    // Default Stream (NULL)
    // ============================================
    // Children on default stream serialize with each other
    // but are async with parent
    
    childA<<<1, 1>>>(data);  // On implicit NULL stream
    childB<<<1, 1>>>(data);  // Waits for childA
    
    cudaDeviceSynchronize();  // Wait for both
    printf("After default stream children: %d\n", *data);
    
    // ============================================
    // Named Streams (Device-Side)
    // ============================================
    cudaStream_t stream1, stream2;
    cudaStreamCreateWithFlags(&stream1, cudaStreamNonBlocking);
    cudaStreamCreateWithFlags(&stream2, cudaStreamNonBlocking);
    
    // Concurrent children on different streams
    childA<<<1, 1, 0, stream1>>>(data);
    childB<<<1, 1, 0, stream2>>>(data);
    
    // Sync specific streams
    cudaStreamSynchronize(stream1);
    cudaStreamSynchronize(stream2);
    
    printf("After named stream children: %d\n", *data);
    
    cudaStreamDestroy(stream1);
    cudaStreamDestroy(stream2);
}

int main() {
    int* d_data;
    cudaMalloc(&d_data, sizeof(int));
    cudaMemset(d_data, 0, sizeof(int));
    
    parentKernel<<<1, 1>>>(d_data);
    cudaDeviceSynchronize();
    
    cudaFree(d_data);
    return 0;
}
```

---

## Part 5: Synchronization Patterns

### 🔷 CUDA C++ When to Sync (Primary)

```cpp
// sync_patterns.cu - Synchronization patterns

// Pattern 1: Sync before reading child results
__global__ void pattern1(float* data) {
    childKernel<<<1, 32>>>(data);
    
    cudaDeviceSynchronize();  // REQUIRED!
    
    float result = data[0];  // Safe to read
}

// Pattern 2: Fire and forget (parent returns)
__global__ void pattern2(float* data) {
    childKernel<<<1, 32>>>(data);
    
    // NO sync - parent returns immediately
    // Child continues running
    // Result visible to HOST after host sync
}

// Pattern 3: Implicit sync at parent completion
// When parent kernel ends, all children are synced
// before control returns to host

__global__ void parentFireForget(float* data) {
    for (int i = 0; i < 10; i++) {
        childKernel<<<1, 32>>>(data + i * 32);
        // No explicit sync
    }
    // When parent ends, runtime syncs all children
}

// Host code:
// parentFireForget<<<1, 1>>>(data);
// cudaDeviceSynchronize();  // Waits for parent AND all children
```

### 🔷 CUDA C++ Error Checking (Primary)

```cpp
__global__ void errorCheckingPattern(float* data) {
    // Launch child
    childKernel<<<1000000, 1024>>>(data);  // Might fail!
    
    // Check for launch errors (device-side)
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("Launch error: %s\n", cudaGetErrorString(err));
        return;
    }
    
    // Sync and check for execution errors
    err = cudaDeviceSynchronize();
    if (err != cudaSuccess) {
        printf("Execution error: %s\n", cudaGetErrorString(err));
    }
}
```

---

## Exercises

### Exercise 1: Simple Recursion
Write a kernel that recursively launches itself (with depth limit).

### Exercise 2: Fan-Out Pattern
One parent thread launches multiple children that process different data.

### Exercise 3: Memory Communication
Parent writes config to global memory, child reads it and writes results.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│           DYNAMIC PARALLELISM BASICS                    │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Launch from GPU:                                       │
│  childKernel<<<blocks, threads>>>(args);                │
│                                                         │
│  Synchronization:                                       │
│  cudaDeviceSynchronize();  // Device-side               │
│                                                         │
│  Compile Flag:                                          │
│  nvcc -rdc=true ...                                     │
│                                                         │
│  Memory Rules:                                          │
│  • Global: visible after sync                           │
│  • Local/Shared: NOT visible to child                   │
│  • Use __threadfence() for visibility                   │
│                                                         │
│  Nesting Limit: 24 levels                               │
│  Compute Capability: 3.5+                               │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## Next: Day 4 - Nested Kernel Patterns